来源：天池比赛论坛上选手的分享
__自己一个人蒙头搞了一段时间，基本能想到的都搞了。这版里就用了Focal loss和fast test的embeddings，也没啥新奇的东西。
线下embeddings concat有挺大的提升的，但是我没提交试试。现在写好了HAN的版本，线下一般般就没提交看看，还有几个版本的self-attention和Multi-Head attention，但是感觉没啥用，也没去跑。__

__一直没有提升搞得我有点崩，接下来试过胶囊网络还没进步就不打算搞了。__

__另外刚开始跑bert的时候，就发现0、1很难区分，但是传统的gbdt好像效果还行，简单的CNN连接lgb就能达到我自己Bert模型的效果。___

__还有今天肉眼扫了一遍数据集，发现了一些奇怪的数据，就正好更新了一下我的`strip_all_entities`函数,效果怎么样我也没来得及试。__

In [14]:
import os, re
import numpy as np
import pandas as pd
from collections import Counter

import nltk
from gensim.models import Word2Vec

import tensorflow as tf
import keras 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from keras.models import Sequential
#from keras import Sequential
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import layers
from tensorflow.keras.layers import Input, LSTM, Embedding, Dropout, Bidirectional, Dense
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, BinaryAccuracy
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.math import rint, reduce_sum, divide, exp, subtract, greater_equal
from tensorflow.math import round as rounding
from tensorflow.random import set_seed
from tensorflow import cast, constant
from tensorflow.nn import softmax


# !pip install plot-keras-history
# from plot_keras_history import plot_history

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score as acu
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

seed = 2022
np.random.seed(seed)
set_seed(seed)

Num GPUs Available:  1


In [15]:
from keras.engine import keras_tensor

ImportError: cannot import name 'keras_tensor' from 'keras.engine' (C:\software\anaconda\envs\gpu\lib\site-packages\keras\engine\__init__.py)

In [16]:
import tensorflow_addons as tfa

ImportError: cannot import name 'keras_tensor' from 'keras.engine' (C:\software\anaconda\envs\gpu\lib\site-packages\keras\engine\__init__.py)

### Load the data


In [17]:
# 加载文本数据
TAIL_VALUES = 10

sel_data = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_sel_log_dataset.csv')
sel_data2 = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_sel_log_dataset_a.csv')
sel_data = pd.concat([sel_data,sel_data2]).sort_values(by=['sn', 'time'], ignore_index=True)
sel_data['time'] = pd.to_datetime(sel_data['time'])
sel_data['week'] = sel_data['time'].dt.isocalendar().week
sel_data = sel_data.groupby(['sn']).tail(TAIL_VALUES)

sel_data2.sort_values(by=['sn', 'time'], ignore_index=True, inplace=True)
sel_data2['time'] = pd.to_datetime(sel_data2['time'])
sel_data2['week'] = sel_data2['time'].dt.isocalendar().week
print(sel_data.shape)
print(sel_data2.shape)

# 加载标签数据
train_label = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_train_label_dataset.csv')
train_label2 = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_train_label_dataset_s.csv')
train_label = pd.concat([train_label, train_label2]).sort_values(by=['sn', 'fault_time'],ignore_index=True)
train_label.drop_duplicates(subset=['sn','fault_time','label'],inplace=True)
train_label['fault_time'] = pd.to_datetime(train_label['fault_time'])
train_label['week'] = train_label['fault_time'].dt.isocalendar().week
print(train_label.shape)

# 加载测试数据
test_data = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/preliminary_submit_dataset_a.csv')
test_data['fault_time'] = pd.to_datetime(test_data['fault_time'])
test_data = test_data.sort_values(by=['sn', 'fault_time'],ignore_index=True)
test_data['week'] = test_data['fault_time'].dt.isocalendar().week
print(test_data.shape)

(103505, 5)
(10991, 5)
(16604, 4)
(3011, 3)


In [18]:
%%time

# 构造训练集
train = pd.merge(sel_data, train_label,on=['sn','week'],how='left')
train = train.query('time <= fault_time')
train.drop_duplicates(subset=['sn','time','msg','fault_time'],inplace=True)
train['text'] = train.groupby(['sn','fault_time','week'])['msg'].transform(lambda x : '.'.join(x))
train['text'] = train['text'].str.lower()
train.drop(['msg','time','week'],axis=1,inplace=True)

train.drop_duplicates(subset=['sn','fault_time','text'],inplace=True)
train.reset_index(inplace=True,drop = True)
train['label'] = train['label'].astype(int)
print(train.label.value_counts())

# 构造测试集
test = pd.merge(test_data, sel_data2,on=['sn','week'],how="left")
test = test.query('time <= fault_time')
test.drop_duplicates(subset=['sn','time','msg','fault_time'],inplace=True)
test['text'] = test.groupby(['sn','fault_time','week'])['msg'].transform(lambda x : '.'.join(x))
test['text'] = test['text'].str.lower()
test.drop(['msg','time','week'],axis=1,inplace=True)
test.drop_duplicates(subset=['sn','fault_time','text'],ignore_index=True,inplace=True)

# 补充测试集
sn_list = ['05aceb8f3b0f', '0e074cf16c7d', '287cfc92991d', '3845fe1dbef9', '82a187bcfa02', 'b64f4cefbbc6', 'd7ca15888043']
supplement_test = test_data.query("sn in @sn_list").copy()
supplement_sel_data = sel_data2.query("sn in @sn_list").copy()

supplement_test = pd.merge(supplement_test, supplement_sel_data,on=['sn'],how="left")
supplement_test['text'] = supplement_test.groupby(['sn','fault_time'])['msg'].transform(lambda x : '.'.join(x))
supplement_test['text'] = supplement_test['text'].str.lower()
supplement_test.drop(['msg','time','week_x','week_y'], axis=1,inplace=True)
supplement_test.drop_duplicates(subset=['sn','fault_time','text'],ignore_index=True,inplace=True)

test = pd.concat([test, supplement_test]).sort_values(by=['sn', 'fault_time'],ignore_index=True)
test.reset_index(inplace=True,drop = True)
print('\n',test.shape)

2    8201
1    3234
3    2310
0    1392
Name: label, dtype: int64

 (3011, 4)
Wall time: 2.02 s


In [19]:
# train_two = train.query("label == 2").copy()
# train_two = train_two[-4000:].copy()
# train_other = train.query("label != 2").copy()
# train = pd.concat([train_other,train_two])
# train.reset_index(inplace=True,drop = True)
# print(train.label.value_counts())

In [20]:
def strip_all_entities(text):
    text = re.sub('锛�','', text)
    text = re.sub('锟絋',' ', text)
    text = re.sub('锟�','', text)
    text = re.sub('\|',',', text)
    text = re.sub('s4/s5:', 'ss_one_state', text)
    text = re.sub('s0/g0:', 'sg_one_state', text)
    text = re.sub('s5/g2:', 'sg_two_state', text)
    text = re.sub('aa17.{22}','', text)
    text = re.sub('000000','a Special tags', text)
    text = re.sub('\d{4}\w\d\w\d{5}','Asserted oem record', text)
    return text
train['text'] = train['text'].apply(lambda x: strip_all_entities(x))
test['text'] = test['text'].apply(lambda x: strip_all_entities(x))

In [21]:
X = train['text']
y = train['label']

# 欠采样 -------------------------------------------
# from imblearn.under_sampling import RandomUnderSampler
# undersample = RandomUnderSampler(sampling_strategy={2:2398,1:3380,3:2398,0:1475})
# X_train, y_train = undersample.fit_resample(np.array(X).reshape(-1, 1), np.array(y).reshape(-1, 1))
# train_os = pd.DataFrame(list(zip([x[0] for x in X_train], y_train)), columns = ['text', 'label'])
# X = train_os['text'].values
# y = train_os['label'].values
# -------------------------------------------------

NUM_CLASSES = 4
MAX_LEN = 110
EMBEDDING_DIM = 200

sentences_train,sentences_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=11) # ,stratify = train_all['label']
print(Counter(y_train))

y_train = tf.keras.utils.to_categorical(y_train, num_classes = NUM_CLASSES)
y_val = tf.keras.utils.to_categorical(y_val, num_classes = NUM_CLASSES)

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(sentences_train)
X_train = tokenizer.texts_to_sequences(sentences_train)
X_val = tokenizer.texts_to_sequences(sentences_val)

# Adding 1 because of  reserved 0 index
vocab_size = len(tokenizer.word_index) + 1 # (in case of pre-trained embeddings it's +2)                         

X_train = pad_sequences(X_train, padding='post', maxlen=MAX_LEN)
X_val = pad_sequences(X_val, padding='post', maxlen=MAX_LEN)


print("Padded and Tokenized Training Sequence".format(),X_train.shape)
print("Target Training Values Shape".format(),y_train.shape)
print("_____________________________________________")
print("Padded and Tokenized Validation Sequence".format(),X_val.shape)
print("Target Validatation Values Shape".format(),y_val.shape)

X_test = tokenizer.texts_to_sequences(test['text'])
X_test = pad_sequences(X_test, padding='post', maxlen=MAX_LEN)

Counter({2: 6548, 1: 2598, 3: 1846, 0: 1117})
Padded and Tokenized Training Sequence (12109, 110)
Target Training Values Shape (12109, 4)
_____________________________________________
Padded and Tokenized Validation Sequence (3028, 110)
Target Validatation Values Shape (3028, 4)


### Fasttext embeddings

In [22]:
# 额外的语料库
addition_corpus = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata/additional_sel_log_dataset.csv')
addition_corpus = addition_corpus.sort_values(by=['sn', 'time'], ignore_index=True)
addition_corpus = addition_corpus.groupby(['sn']).tail(9)
addition_corpus['text'] = addition_corpus.groupby(['sn'])['msg'].transform(lambda x : ''.join(x))
addition_corpus['text'] = addition_corpus['text'].str.lower()
addition_corpus.drop(['msg'],axis=1,inplace=True)
addition_corpus.drop_duplicates(subset=['sn', 'text'],ignore_index=True,inplace=True)

training_corpus = train['text']   # train_all
print(len(training_corpus))
training_corpus = training_corpus.append(addition_corpus['text'],ignore_index=True)
print(len(training_corpus))
training_corpus = pd.DataFrame(training_corpus)

15137
26786


In [23]:
%%time
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from textblob import TextBlob
corpus = []
for row in zip(*training_corpus.to_dict("list").values()):
    zen = TextBlob(row[0]).words
    corpus.append(zen)

Wall time: 17.2 s


In [24]:
%%time
from gensim.models import FastText
# model = FastText(vector_size=200, window=8, min_count=0)  # instantiate
# model.build_vocab(corpus_iterable = corpus)
# model.train(corpus_iterable=corpus, total_examples=len(a), epochs=10)
model_fasttext = FastText(vector_size=EMBEDDING_DIM, window=8, min_count=1, sentences=corpus, epochs=10,sg = 1,hs = 1,sample = 1e-3, ns_exponent = 0.25) 

2022-03-31 11:26:01,813 : INFO : collecting all words and their counts
2022-03-31 11:26:01,814 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2022-03-31 11:26:01,868 : INFO : PROGRESS: at sentence #10000, processed 393643 words, keeping 923 word types
2022-03-31 11:26:01,938 : INFO : PROGRESS: at sentence #20000, processed 887111 words, keeping 1333 word types
2022-03-31 11:26:01,994 : INFO : collected 2612 word types from a corpus of 1300406 raw words and 26786 sentences
2022-03-31 11:26:01,995 : INFO : Creating a fresh vocabulary
2022-03-31 11:26:02,012 : INFO : FastText lifecycle event {'msg': 'effective_min_count=1 retains 2612 unique words (100.0%% of original 2612, drops 0)', 'datetime': '2022-03-31T11:26:02.012610', 'gensim': '4.0.1', 'python': '3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'prepare_vocab'}
2022-03-31 11:26:02,013 : INFO : FastText lifecycle event {'msg': 'effect

2022-03-31 11:26:38,012 : INFO : EPOCH 6 - PROGRESS: at 18.11% examples, 74259 words/s, in_qsize 5, out_qsize 0
2022-03-31 11:26:39,030 : INFO : EPOCH 6 - PROGRESS: at 36.10% examples, 79250 words/s, in_qsize 5, out_qsize 0
2022-03-31 11:26:40,103 : INFO : EPOCH 6 - PROGRESS: at 51.34% examples, 76177 words/s, in_qsize 5, out_qsize 0
2022-03-31 11:26:41,123 : INFO : EPOCH 6 - PROGRESS: at 65.18% examples, 77568 words/s, in_qsize 5, out_qsize 0
2022-03-31 11:26:42,238 : INFO : EPOCH 6 - PROGRESS: at 78.35% examples, 77285 words/s, in_qsize 5, out_qsize 0
2022-03-31 11:26:43,293 : INFO : EPOCH 6 - PROGRESS: at 92.00% examples, 78228 words/s, in_qsize 6, out_qsize 0
2022-03-31 11:26:43,805 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-31 11:26:43,848 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-31 11:26:43,873 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-31 11:26:43,874 : INFO : EPOCH - 6 : training o

Wall time: 1min 9s


In [16]:
fasttext_embedding_idx = {}
for index, word in enumerate(model_fasttext.wv.index_to_key):
    embeddings = np.asarray(model_fasttext.wv[word])
    fasttext_embedding_idx[word] = embeddings

In [17]:
iv = 0
oov = 0

embedding_idx = fasttext_embedding_idx # swap between embeddings

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_idx.get(word)
    if embedding_vector is not None:
        iv += 1
        # words not found in the embedding space are all zeros
        embedding_matrix[i] = embedding_vector
    else: 
        oov += 1
        
print('%i tokens in vocab, %i tokens out of vocab' % (iv, oov))

655 tokens in vocab, 182 tokens out of vocab


In [18]:
embedding_layer = Embedding(
    vocab_size,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    trainable=False
    )

In [22]:
optimizer = Adamax(learning_rate=0.002, decay=0.0005, clipvalue=10)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics=[
       tf.keras.metrics.Precision(),
       tf.keras.metrics.Recall(),
       tfa.metrics.F1Score(num_classes=NUM_CLASSES,average='macro',threshold=0.5)]

def FocalLoss(target, input):
    gamma = 3. # Hyperparameter that can be tuned.

    input = tf.cast(input, tf.float32)    
    max_val = K.clip(-input, 0, 1)
    loss = input - input * target + max_val + K.log(K.exp(-max_val) + K.exp(-input - max_val))
    invprobs = tf.math.log_sigmoid(-input * (target * 2.0 - 1.0))
    loss = K.exp(invprobs * gamma) * loss    
    return K.mean(K.sum(loss, axis=1))

### CNN

In [26]:
int_sequences_input = Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
drop = Dropout(0.3)(embedded_sequences)
x = layers.Conv1D(256, 5, activation="relu")(drop)
x = layers.MaxPooling1D()(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D()(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(64,activation='relu')(x)
preds = layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(int_sequences_input, preds)

model.summary()
model.compile(optimizer=optimizer,loss=FocalLoss, metrics=metrics) 

AttributeError: module 'keras.layers' has no attribute 'Conv1D'

In [ ]:
history=model.fit(X_train, y_train,epochs=10, validation_data=(X_val, y_val),batch_size=64)

In [ ]:
preds = model.predict(X_test,verbose = 1)
pred_classes = np.argmax(preds, axis = 1)
print(Counter(pred_classes))
submit = test.copy()
submit['label'] = pred_classes
submit['label'].value_counts()

### TextCNN

In [ ]:
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

def model_cnn():
    filter_sizes = [1,2,3,5]
    num_filters = 36

    inp = Input(shape=(MAX_LEN,))
    x = embedding_layer(inp)
    x = Reshape((MAX_LEN, EMBEDDING_DIM, 1))(x)

    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], EMBEDDING_DIM),
                                     kernel_initializer='he_normal', activation='relu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(MAX_LEN - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    outp = Dense(3, activation="softmax")(z)

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=FocalLoss, optimizer=optimizer, metrics=metrics)
    
    return model
model = model_cnn()
model.summary()

In [ ]:
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)
for e in range(8):
    model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_val, y_val), 
              verbose = 1, callbacks = [early_stop])
pred_val_y = model.predict([X_val], batch_size=128, verbose=0)
pred_test_y = model.predict([X_test], batch_size=128, verbose=0)

In [ ]:
pred_classes = np.argmax(pred_test_y, axis = 1)
submit = test.copy()
submit['label'] = pred_classes
submit['label'].value_counts()

### Bi-GRU + Bi-LSTM 

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
# from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import BatchNormalization, Conv1D, MaxPooling1D

# check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
#                               save_best_only = True, mode = "min")
# ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (MAX_LEN,))
    x = embedding_layer(inp)
    x1 = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x1)
    x = Conv1D(int(units/2), kernel_size = 7, padding = "valid", kernel_initializer = "he_uniform")(x)
    
    y = Bidirectional(LSTM(units, return_sequences = True))(x1)
    y = Conv1D(int(units/2), kernel_size = 7, padding = "valid", kernel_initializer = "he_uniform")(y)
    
    avg_pool1 = GlobalAveragePooling1D()(x)
    max_pool1 = GlobalMaxPooling1D()(x)
    
    avg_pool2 = GlobalAveragePooling1D()(y)
    max_pool2 = GlobalMaxPooling1D()(y)
    
    
    x = concatenate([avg_pool1, max_pool1, avg_pool2, max_pool2])

    x = Dense(NUM_CLASSES, activation = "softmax")(x)
    model = Model(inputs = inp, outputs = x)
    model.summary()
    model.compile(loss = FocalLoss, optimizer = Adamax(learning_rate = lr, decay = lr_d), metrics = metrics)
    history = model.fit(X_train, y_train, batch_size = 128, epochs = 30, validation_data = (X_val, y_val), 
                        verbose = 1, callbacks = [ early_stop])
    return model
    
model = build_model(lr = 2e-3, lr_d = 0.0005, units = 128, dr = 0.2)
preds = model.predict(X_test, batch_size = 64, verbose = 1)

In [ ]:
pred_classes = np.argmax(preds, axis = 1)
submit = test.copy()
submit['label'] = pred_classes
submit['label'].value_counts()